In [1]:
!pip3 install langchain_community
!pip3 install pandas



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.llms import Ollama
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

C:\Users\vouth\AppData\Local\Temp\ipykernel_32664\3421170792.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


'\nThe first man on the moon was Neil Armstrong. He stepped foot on the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" as he took his first steps on the lunar surface.'

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nOf course! Here are the categories for each expense you provided:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* Taxi Utrecht - Transportation\n* Ministerie van Justitie en Veiligheid - Government\n* Etos AMSTERDAM NLD - Grocery\n* Bistro Bar Amsterdam - Food and Drink'

### Read transaction data

In [5]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [6]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [7]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [30]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 50))
index_list

[0, 49]

In [31]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [32]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Sure! Here are the categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. ISS Catering Services De Meern - Food', '3. Taxi Utrecht - Transportation', '4. Etos AMSTERDAM NLD - Groceries', '5. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Sure! Here are the categories for each expense:,Sure! Here are the categories for each expense:,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. ISS Catering Services De Meern - Food,2. ISS Catering Services De Meern,Food
5,3. Taxi Utrecht - Transportation,3. Taxi Utrecht,Transportation
6,4. Etos AMSTERDAM NLD - Groceries,4. Etos AMSTERDAM NLD,Groceries
7,5. Bistro Bar Amsterdam - Dining,5. Bistro Bar Amsterdam,Dining


In [33]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Of course! Here are the transactions with appropriate categories:', '', 'Transaction 1 - Entertainment', '', '* Spotify AB by Adyen ($10.00)', '', 'Transaction 2 - Sport', '', '* Beta Boulders Ams Amsterdam Nld ($25.00)', '', 'Transaction 3 - Travel', '', '* Flight from NYC to LA ($300.00)', '', 'Transaction 4 - Eating Out', '', '* Italian Restaurant in Manhattan ($60.00)', '', 'Transaction 5 - Groceries', '', '* Whole Foods Gift Card ($100.00)', '', 'Transaction 6 - Clothing', '', '* New Dress from Nordstrom ($150.00)', '', 'Transaction 7 - Electronics', '', '* New Laptop from Best Buy ($1,000.00)', '', 'Transaction 8 - Home Improvement', '', '* Home Depot Gift Card ($200.00)', '', 'Transaction 9 - Pet Expenses', '', '* Vet Visit for Fluffy ($75.00)', '', 'Transaction 10 - Miscellaneous', '', '* Uber Rides in LA ($40.00)', '', 'Transaction 11 - Entertainment', '', '* Ticketmaster Gift Card ($200.00)', '', 'Transaction 12 - Travel', '', '* Hotel Stay in Miami ($150.00)', '', 'Transac

In [34]:
import pandas as pd

# Simulate 30 unique transactions
unique_transactions = [
    f"Transaction{i} - Category{i % 5}" for i in range(1, 50)  # 5 categories cycling
]

# Simulate index_list to group transactions in chunks of 10
index_list = list(range(0, len(unique_transactions) + 1, 10))  # [0, 10, 20, 30]

categories_df_all = pd.DataFrame()

# Simulated categorize_transactions function
def categorize_transactions(transaction_names, llm):
    # Simulate response in the expected format
    response = transaction_names.split(",")
    categories_df = pd.DataFrame({'Transaction vs category': response})
    return categories_df

# Process the transactions
for i in range(0, len(index_list) - 1):
    # Get transactions in the current range
    transaction_names = unique_transactions[index_list[i]:index_list[i + 1]]
    transaction_names = ','.join(transaction_names)
    
    # Call categorize_transactions and handle errors
    try:
        categories_df = categorize_transactions(transaction_names, llm=None)  # Pass 'llm' as needed
        # Filter and split valid rows
        categories_df = categories_df[categories_df['Transaction vs category'].str.contains(' - ', na=False)]
        categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
        # Concatenate to the master dataframe
        categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
    except Exception as e:
        print(f"Error processing transactions in range {i}-{i+1}: {e}")

# Display the final output
print(categories_df_all)


      Transaction vs category    Transaction   Category
0    Transaction1 - Category1   Transaction1  Category1
1    Transaction2 - Category2   Transaction2  Category2
2    Transaction3 - Category3   Transaction3  Category3
3    Transaction4 - Category4   Transaction4  Category4
4    Transaction5 - Category0   Transaction5  Category0
5    Transaction6 - Category1   Transaction6  Category1
6    Transaction7 - Category2   Transaction7  Category2
7    Transaction8 - Category3   Transaction8  Category3
8    Transaction9 - Category4   Transaction9  Category4
9   Transaction10 - Category0  Transaction10  Category0
10  Transaction11 - Category1  Transaction11  Category1
11  Transaction12 - Category2  Transaction12  Category2
12  Transaction13 - Category3  Transaction13  Category3
13  Transaction14 - Category4  Transaction14  Category4
14  Transaction15 - Category0  Transaction15  Category0
15  Transaction16 - Category1  Transaction16  Category1
16  Transaction17 - Category2  Transaction17  Ca

In [35]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Transaction1 - Category1,Transaction1,Category1
1,Transaction2 - Category2,Transaction2,Category2
2,Transaction3 - Category3,Transaction3,Category3
3,Transaction4 - Category4,Transaction4,Category4
4,Transaction5 - Category0,Transaction5,Category0
5,Transaction6 - Category1,Transaction6,Category1
6,Transaction7 - Category2,Transaction7,Category2
7,Transaction8 - Category3,Transaction8,Category3
8,Transaction9 - Category4,Transaction9,Category4
9,Transaction10 - Category0,Transaction10,Category0


In [36]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [15]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Category1', 'Category2', 'Category3', 'Category4', 'Category0'],
      dtype=object)

In [41]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [42]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
0,Transaction1 - Category1,Transaction1,Category1
1,Transaction2 - Category2,Transaction2,Category2
2,Transaction3 - Category3,Transaction3,Category3
3,Transaction4 - Category4,Transaction4,Category4
4,Transaction5 - Category0,Transaction5,Category0
5,Transaction6 - Category1,Transaction6,Category1
6,Transaction7 - Category2,Transaction7,Category2
7,Transaction8 - Category3,Transaction8,Category3
8,Transaction9 - Category4,Transaction9,Category4
9,Transaction10 - Category0,Transaction10,Category0


In [43]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [45]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)

In [20]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (EUR),Category,Year,Month,Month Name
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,2023,12,Dec
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,2023,12,Dec
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,2023,12,Dec
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,2023,12,Dec
4,2023-12-29,Selling Paintings,Income,13.63,NaN,2023,12,Dec
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,2023,12,Dec
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,2023,12,Dec
7,2023-12-22,Consulting,Income,541.57,NaN,2023,12,Dec
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,2023,12,Dec
9,2023-12-20,Consulting,Income,2641.93,NaN,2023,12,Dec


In [21]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

In [22]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [23]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

In [24]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [25]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

In [26]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig



In [27]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022